# ATLAS top mass combination (7 + 8 TeV)

In this notebook, the results of [ATLAS-CONF-2017-071](https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/CONFNOTES/ATLAS-CONF-2017-071/) are reproduced.
The CONF note reports the measurement of the top mass in 
the lepton + jets channel at $\sqrt{s}=8$ TeV, before a combination 
with previous measurements is performed.

In [1]:
import numpy as np
import pandas as pd

from blue import Blue

The data from the previous three measurements is 
store in `data/atlas_top_mass_dilep8.csv` as it was used in a previous combination and 
the data from this measurement is store in `data/atlas_top_mass_ljets8.csv`. We read this
in and combined them into a single dataframe.

In [2]:
dilep = pd.read_csv('data/atlas_top_mass_dilep8.csv', index_col='Name')
ljets = pd.read_csv('data/atlas_top_mass_ljets8.csv', index_col='Name')
df = pd.concat([dilep, ljets])
df.T

Name,ljets_7,dilep_7,dilep_8,ljets_8
Result,172.33,173.79,172.99,172.08
Stats,0.75,0.54,0.41,0.39
Method,0.11,0.09,0.05,0.13
SignalMC,0.22,0.26,0.09,0.16
Hadronisation,0.18,0.53,0.22,0.15
IFSR,0.32,0.47,0.23,0.08
UE,0.15,0.05,0.10,0.08
CR,0.11,0.14,0.03,0.19
PDF,0.25,0.11,0.05,0.09
BackNorm,0.10,0.04,0.03,0.08


We can see the results of each measurement and their uncertainties. These can be compared with Table 4 in ATLAS-CONF-2017-071. For some reason, the numbers in the dilepton 8 TeV column are slightly different to those in the [paper](https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/TOPQ-2016-03/) (I'm not sure why) so we'll update them.

In [3]:
df.loc['dilep_8'].loc['FakeShape'] = 0.07
df.loc['dilep_8'].loc['btagging'] = 0.04
df.T

Name,ljets_7,dilep_7,dilep_8,ljets_8
Result,172.33,173.79,172.99,172.08
Stats,0.75,0.54,0.41,0.39
Method,0.11,0.09,0.05,0.13
SignalMC,0.22,0.26,0.09,0.16
Hadronisation,0.18,0.53,0.22,0.15
IFSR,0.32,0.47,0.23,0.08
UE,0.15,0.05,0.10,0.08
CR,0.11,0.14,0.03,0.19
PDF,0.25,0.11,0.05,0.09
BackNorm,0.10,0.04,0.03,0.08


The next step is to define the correlations. 
In the CONF note the columns have swapped order w.r.t. the dilepton paper combination, which changes
the order that the correlations are inputed so there is a slight mismatch between what is below and what is in the paper $(\rho_{03} \leftrightarrow \rho_{13})$.

In [4]:
correlations = {
    'Stats': 0,
    'Method': 0,
    'SignalMC': 1,
    'Hadronisation': [1, 1, -1, 1, -1, -1],
    'IFSR': [-1, -1, -1, 1, 1, 1],
    'UE': [-1, -1, -1, 1, 1, 1],
    'CR': [-1, -1, 1, 1, -1, -1],
    'PDF': [0.57, -0.29, 0.72, 0.03, 0.72, -0.48],
    'BackNorm': [1, 0.23, -0.74, 0.23, -0.77, -0.06],
    'WZShape': 0,
    'FakeShape': [0.23, 0.20, 0, -0.08, 0, 0],
    'JES': [-0.23, 0.06, -0.29, 0.35, 0.18, -0.54],
    'btolightJES': 1,
    'JER': [-1, 0, 0, 0, 0, 0.22],
    'JetRecoEff': 1,
    'JVF': [-1, 1, 1, -1, -1, 1],
    'btagging': [-0.77, 0, 0, 0, 0, -0.23],
    'leptons': [-0.34, -0.52,-0.17, 0.96, -0.08, 0.11],
    'Etmiss': [-0.15, 0.25, 0.22, -0.24, -0.12, 0.97],
    'Pileup': 0,
}

Because this is a bit messy (ideas to make this cleaner are welcome!) we will check that the only key that exists in the data but not in the correlations is the `Result` column.

In [5]:
df.columns ^ correlations

Index(['Result'], dtype='object')

Ok, we entered a correlation for each of the systematic sources so we can now go ahead and perform the combination. To begin with we will use all four measurements, although in the end the CONF note only uses three.

In [6]:
combination = Blue(df, correlations)
combination.combined_result

172.49914976824516

Combining the four measurements gives us a top mass of ~172.5 GeV. 
Let's look at the uncertainties.

In [7]:
uncerts = pd.Series(combination.combined_uncertainties)[df.columns.drop('Result')]
np.round(uncerts, 2)

Stats            0.27
Method           0.06
SignalMC         0.14
Hadronisation    0.06
IFSR             0.07
UE               0.05
CR               0.08
PDF              0.07
BackNorm         0.03
WZShape          0.07
FakeShape        0.03
JES              0.22
btolightJES      0.14
JER              0.11
JetRecoEff       0.03
JVF              0.05
btagging         0.17
leptons          0.09
Etmiss           0.04
Pileup           0.06
dtype: float64

In [8]:
total = np.sqrt((uncerts ** 2).sum())
syst = np.sqrt((uncerts.drop('Stats') ** 2).sum())

and let's print the results out with the uncertainty:

In [9]:
print('Combined result = '
      f'{combination.combined_result:.2f} '
      f'+- {uncerts["Stats"]:.2f} (stat) '
      f'+- {syst:.2f} (syst) GeV')
print('Combined result = '
      f'{combination.combined_result:.2f} '
      f'+- {total:.2f} GeV')

Combined result = 172.50 +- 0.27 (stat) +- 0.42 (syst) GeV
Combined result = 172.50 +- 0.50 GeV


This can be compared to the final point in Figure 8 (a), which shows the results from combining all four measurements. We can also check the compatibility of the measurements:

In [10]:
from scipy.stats import chi2
chi2.sf(*combination.chi2_ndf)

0.77437217885707621

We see a compatibility of 77%. We can also look at the correlations between each individual measurement. We will make this into a pandas.DataFrame to make things a bit clearer and reorder the rows/columns to be in the same order as the CONF note.

In [11]:
reorder_cols = ['dilep_7', 'ljets_7', 'dilep_8', 'ljets_8']
measurement_names = combination.data.index
_ = pd.DataFrame(
    combination.total_correlations, 
    columns=measurement_names,
    index=measurement_names
    ).loc[reorder_cols][reorder_cols]
np.round(_, 2)

Name,dilep_7,ljets_7,dilep_8,ljets_8
Name,,,,
dilep_7,1.00,-0.07,0.52,0.06
ljets_7,-0.07,1.00,0.00,-0.07
dilep_8,0.52,0.00,1.00,-0.19
ljets_8,0.06,-0.07,-0.19,1.00


These correlations can be compared with those given in Table 6 of the CONF note

In [12]:
np.round(pd.Series(
    combination.weights, 
    index=combination.data.index
)[reorder_cols], 2)

Name
dilep_7   -0.02
ljets_7    0.17
dilep_8    0.45
ljets_8    0.40
dtype: float64

Table 6 of the CONF note only shows the weights for "final" combination that doesn't include the 7 TeV dilepton measurement, although it can been seen that the weight given to this measurement is small, and the lepton + jets weights are the same with or without including this.

We can also test the case in which we combine as if looking at two observables, the top mass in the $\ell+$jets channel, and the top mass in the dilepton channel. The results are: 

In [13]:
combination.observables = {
    'ljets': ['ljets_7', 'ljets_8'],
    'dilep': ['dilep_7', 'dilep_8']
}
combination.combined_result

{'dilep': 173.01900964359118, 'ljets': 172.10883942618131}

and we can check the correlations between these two new observables:

In [14]:
combination.observable_correlations[1,0]

-0.14799705467325977

In [15]:
# Here we reset the observables to None (which is a single observable)
combination.observables = None

The final combination does not use the 7 TeV dilepton measurement, we can get this set of measurements as follows:

In [16]:
final_comb = combination[['ljets_7', 'dilep_8', 'ljets_8']]
final_comb.combined_result

172.51573540994275

We get a slightly different value from the one quoted in the CONF note but generally consistent. Let us look at the uncertainties.

In [17]:
final_uncerts = pd.Series(final_comb.combined_uncertainties)

np.round(final_uncerts[final_comb.data.columns.drop('Result')], 2)

Stats            0.27
Method           0.06
SignalMC         0.14
Hadronisation    0.07
IFSR             0.08
UE               0.05
CR               0.08
PDF              0.07
BackNorm         0.03
WZShape          0.07
FakeShape        0.03
JES              0.22
btolightJES      0.15
JER              0.10
JetRecoEff       0.03
JVF              0.05
btagging         0.17
leptons          0.09
Etmiss           0.04
Pileup           0.06
dtype: float64

These numbers are almost the same as in the CONF note, with only the IFSR and JES systematics being slightly different. Let's print the final combined result and the $\chi^2$ probability of the combination!

In [18]:
final_total = np.sqrt((final_uncerts ** 2).sum())
final_syst = np.sqrt((final_uncerts.drop('Stats') ** 2).sum())
print('Combined result = '
      f'{final_comb.combined_result:.2f} '
      f'+- {uncerts["Stats"]:.2f} (stat) '
      f'+- {syst:.2f} (syst) GeV')
print('Combined result = '
      f'{final_comb.combined_result:.2f} '
      f'+- {total:.2f} GeV')
c2, ndf = final_comb.chi2_ndf
print(f'Chi^2 = {c2:.2f}, NDF = {ndf}, p-value = {chi2.sf(c2, ndf):.2f}')

Combined result = 172.52 +- 0.27 (stat) +- 0.42 (syst) GeV
Combined result = 172.52 +- 0.50 GeV
Chi^2 = 0.49, NDF = 2, p-value = 0.78
